[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/Mix-of-Show-colab/blob/main/Mix_of_Show_fused_single_anime_colab.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/Mix-of-Show
%cd /content/Mix-of-Show

!pip install -q diffusers==0.20.2 transformers accelerate einops omegaconf

!git clone https://huggingface.co/ckpt/MoS-hina-kario-tezuka-anythingv4

import json
import os
import torch
from diffusers import DPMSolverMultistepScheduler
from mixofshow.pipelines.pipeline_edlora import EDLoRAPipeline
pretrained_model_path = '/content/Mix-of-Show/MoS-hina-kario-tezuka-anythingv4'
enable_edlora = True  # True for edlora, False for lora
pipe = EDLoRAPipeline.from_pretrained(pretrained_model_path, scheduler=DPMSolverMultistepScheduler.from_pretrained(pretrained_model_path, subfolder='scheduler'), torch_dtype=torch.float16).to('cuda')
with open(f'{pretrained_model_path}/new_concept_cfg.json', 'r') as fr:
    new_concept_cfg = json.load(fr)
pipe.set_new_concept_cfg(new_concept_cfg)

In [ ]:
TOK = '<hina1> <hina2>'  # the TOK is the concept name when training lora/edlora
prompt = f'a {TOK} in front of mount fuji'
negative_prompt = 'longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality'
image = pipe(prompt, negative_prompt=negative_prompt, height=1024, width=512, num_inference_steps=50, generator=torch.Generator('cuda'), guidance_scale=7.5).images[0]
image.save(f'res.jpg')
image